In [9]:
from six.moves import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread, imshow

In [10]:
def load_pickle(f):
    return  pickle.load(f, encoding='latin1')
    
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [11]:
xTrain, yTrain, xTest, yTest = load_CIFAR10("Datasets/cifar-10-batches-py/")
print("X_train:", xTrain.shape)
print("Y_train:", yTrain.shape)
print("X_test:", xTest.shape)
print("Y_test:", yTest.shape)

X_train: (50000, 32, 32, 3)
Y_train: (50000,)
X_test: (10000, 32, 32, 3)
Y_test: (10000,)


In [12]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# models

In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [14]:
class Cifar10(Dataset):
    def __init__(self, data, label, data_num):
        super(Cifar10, self).__init__()
        self.data_num = data_num
        self.data = data
        self.label = label
        self.MEAN = [125.306918046875, 122.950394140625, 113.86538318359375]
        self.STD = [62.993219278136884, 62.08870764001421, 66.70489964063091]
    
    def __getitem__(self, index):
        # 归一化
        data = self.data[index].astype(np.float32)
        data[:, :, 0] = (data[:, :, 0] - self.MEAN[0]) / self.STD[0]
        data[:, :, 1] = (data[:, :, 1] - self.MEAN[1]) / self.STD[1]
        data[:, :, 2] = (data[:, :, 2] - self.MEAN[2]) / self.STD[2]
        label = self.label[index]
        return data, label
    
    def __len__(self):
        return self.data_num


In [28]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3072, 3072),
            nn.BatchNorm1d(3072),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(3072, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )
    
    def forward(self, data):
        output = self.model(data)
        return output
    

# train and test

In [29]:
NUM_TRAIN = 50000
NUM_TEST = 10000
BATCH_SIZE = 256
LEARNING_RATE = 0.001

trainDataset = Cifar10(xTrain, yTrain, NUM_TRAIN)
testDataset = Cifar10(xTest, yTest, NUM_TEST)
trainLoader = DataLoader(dataset=trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
testLoader = DataLoader(dataset=testDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
model = MLP()
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [30]:
%%time
NUM_EPOCHS = 50
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    test_loss = 0
    train_acc = 0
    test_acc = 0
    model.train()
    for i, (data, label) in enumerate(trainLoader):
        data = Variable(data.view(-1, 3072)).cuda()
        label = Variable(label.view(-1)).cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        train_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        num_correct = (predict == label).sum()
        train_acc += num_correct.data[0]
        optimizer.step()
    model.eval()
    for i, (data, label) in enumerate(testLoader):
        data = Variable(data.view(-1, 3072)).cuda()
        label = Variable(label.view(-1)).cuda()
        output = model(data)
        loss = criterion(output, label)
        test_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        num_correct = (predict == label).sum()
        test_acc += num_correct.data[0]
        
    print('Epoch [%d/%d], Train Loss: %.4f, Train Acc: %.4f, Test Loss: %.4f, Test Acc: %.4f'
            %(epoch+1, NUM_EPOCHS, 
              train_loss / NUM_TRAIN, train_acc / NUM_TRAIN, 
              test_loss / NUM_TEST, test_acc / NUM_TEST))

Epoch [1/50], Train Loss: 0.0069, Train Acc: 0.3698, Test Loss: 0.0060, Test Acc: 0.4548
Epoch [2/50], Train Loss: 0.0061, Train Acc: 0.4448, Test Loss: 0.0056, Test Acc: 0.4820
Epoch [3/50], Train Loss: 0.0057, Train Acc: 0.4763, Test Loss: 0.0054, Test Acc: 0.5038
Epoch [4/50], Train Loss: 0.0055, Train Acc: 0.4967, Test Loss: 0.0053, Test Acc: 0.5140
Epoch [5/50], Train Loss: 0.0053, Train Acc: 0.5145, Test Loss: 0.0052, Test Acc: 0.5249
Epoch [6/50], Train Loss: 0.0052, Train Acc: 0.5239, Test Loss: 0.0050, Test Acc: 0.5352
Epoch [7/50], Train Loss: 0.0050, Train Acc: 0.5402, Test Loss: 0.0050, Test Acc: 0.5436
Epoch [8/50], Train Loss: 0.0049, Train Acc: 0.5515, Test Loss: 0.0049, Test Acc: 0.5503
Epoch [9/50], Train Loss: 0.0048, Train Acc: 0.5613, Test Loss: 0.0049, Test Acc: 0.5531
Epoch [10/50], Train Loss: 0.0047, Train Acc: 0.5729, Test Loss: 0.0048, Test Acc: 0.5606
Epoch [11/50], Train Loss: 0.0046, Train Acc: 0.5829, Test Loss: 0.0048, Test Acc: 0.5611
Epoch [12/50], Trai